In [1]:
import pathlib

import numpy as np
import pandas as pd

import hier
import metrics

In [2]:
exp_dirs = {
    # lr = 0.1
    3: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-07-inat21-trunc/flat-depth-3/',
    4: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-07-inat21-trunc/flat-depth-4/',
    5: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-07-inat21-trunc/flat-depth-5/',
    6: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-07-inat21-trunc/flat-depth-6/',
    7: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-07-inat21-trunc/flat/',

#     # lr = 0.01
#     3: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-07-inat21-trunc/flat-depth-3-lr-0.01/',
#     4: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-07-inat21-trunc/flat-depth-4-lr-0.01/',
#     5: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-07-inat21-trunc/flat-depth-5-lr-0.01/',
#     6: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-07-inat21-trunc/flat-depth-6-lr-0.01/',
#     7: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-03-31-inat21mini/flat_softmax-lr-0.01-b-64-wd-0.0003-ep-20',
}

In [3]:
# Convert from single array to shards that can be streamed.
# For full hierarchy, the probability array is [100000, 10000] > 3GiB.

def convert(pkl_file, npz_file):
    outputs = np.load(pkl_file, allow_pickle=True)
    x = outputs['prob']
    # Divide into shards of around 128MiB.
    num_shards = np.ceil(np.size(x) * x.dtype.itemsize / (2 ** 27))
    np.savez(npz_file, *np.array_split(x, num_shards))

for name, exp_dir in exp_dirs.items():
    exp_dir = pathlib.Path(exp_dir)
    npz_file = exp_dir / 'predictions/prob-epoch-0020.npz'
    pkl_file = exp_dir / 'predictions/full-output-epoch-0020.pkl'
    if pkl_file.exists() and not npz_file.exists():
        convert(pkl_file=pkl_file, npz_file=npz_file)

In [4]:
with open('resources/hierarchy/inat21.csv') as f:
    tree, names = hier.make_hierarchy_from_edges(hier.load_edges(f))

In [5]:
subtrees = {}
node_subsets = {}
projections = {}

for depth in exp_dirs:
    if depth == 7:
        subtree = tree
        node_subset = np.arange(tree.num_nodes())
        projection = np.arange(tree.num_nodes())
    else:
        subtree_file = f'resources/subtree/inat21_max_depth_{depth}.csv'
        with open(subtree_file) as f:
            subtree, subtree_names = hier.make_hierarchy_from_edges(hier.load_edges(f))
        node_subset = hier.find_subset_index(names, subtree_names)
        projection = hier.find_projection(tree, node_subset)

    subtrees[depth] = subtree
    node_subsets[depth] = node_subset
    projections[depth] = projection

In [6]:
acc = {}

for train_depth in exp_dirs:
    output_file = pathlib.Path(exp_dirs[train_depth]) / 'predictions/output-epoch-0020.pkl'
    outputs = np.load(output_file, allow_pickle=True)

    # To use 'leaf' predictions, we should load the full probability vector
    # and do max-leaf inference within the sub-tree.
    # On the other hand, 'majority' inference can be projected post-hoc.
    pred_subtree = outputs['pred']['majority']
    gt_subtree = outputs['gt']
    train_node_subset = node_subsets[train_depth]
    pred = train_node_subset[pred_subtree]
    gt = train_node_subset[gt_subtree]

    for test_depth in exp_dirs:
        if test_depth > train_depth:
            continue
        # Project ground-truth to test
        project = projections[test_depth]
        gt_proj = project[gt]
        pred_proj = project[pred]
        acc[train_depth, test_depth] = np.mean(gt_proj == pred_proj)

In [7]:
pd.Series(acc).unstack()

,3,4,5,6,7
3,0.78844,NaN,NaN,NaN,NaN
4,0.81713,0.48301,NaN,NaN,NaN
5,0.83108,0.52361,0.35589,NaN,NaN
6,0.83270,0.55167,0.40487,0.26763,NaN
7,0.83447,0.57058,0.42960,0.30600,0.22784


In [8]:
# Obtain *per-class* accuracy at each depth.

per_class = {}

for train_depth in exp_dirs:
    output_file = pathlib.Path(exp_dirs[train_depth]) / 'predictions/output-epoch-0020.pkl'
    outputs = np.load(output_file, allow_pickle=True)

    # To use 'leaf' predictions, we should load the full probability vector
    # and do max-leaf inference within the sub-tree.
    # On the other hand, 'majority' inference can be projected post-hoc.
    pred_subtree = outputs['pred']['majority']
    gt_subtree = outputs['gt']
    train_node_subset = node_subsets[train_depth]
    pred = train_node_subset[pred_subtree]
    gt = train_node_subset[gt_subtree]

    for test_depth in exp_dirs:
        if test_depth > train_depth:
            continue
        # Project ground-truth to test
        project = projections[test_depth]
        gt_proj = project[gt]
        pred_proj = project[pred]

        is_correct = (gt_proj == pred_proj)
        per_class[train_depth, test_depth] = np.array([
            np.mean(is_correct[gt_proj == y])
            for y in subtrees[test_depth].leaf_subset()
        ])

    del outputs

In [9]:
# Check whether it is ever advantageous to train at a *coarser* level.
# Consider training at two different levels A < B (i.e. B is deeper).
# Compare the per-class accuracy at some level <= A.

# Compute (level A) - (level B).
# Greater positive numbers mean that coarse training is better.
percentiles = [0.5, 0.85, 0.95]

rows = {}

for depth_a in exp_dirs:
    for depth_b in exp_dirs:
        if not depth_a < depth_b:
            continue
        for test_depth in exp_dirs:
            if not test_depth <= depth_a:
                continue
            rows[test_depth, depth_a, depth_b] = pd.DataFrame([
                per_class[depth_a, test_depth] - per_class[depth_b, test_depth]
            ]).T.describe(percentiles).T

table = pd.concat(rows).sort_index()
table = table.droplevel(3)
table.index = table.index.set_names(['test', 'a', 'b'])
del table['count']
# table.format('{:+.1%}')

def highlight(s):
    if s.name == 'std':
        return None
    return ['color: green;' if x > 0 else 'color: black;' for x in s]

table.style.apply(highlight).format('{:+.1%}')

In [10]:
# Same thing but always evaluate at coarse level.
# (This is just less information to interpret.)
# Greater positive numbers mean coarser is better.

percentiles = [0.5, 0.85, 0.95]

rows = {}
for test_depth in exp_dirs:
    for train_depth in exp_dirs:
        if train_depth <= test_depth:
            continue
        rows[test_depth, train_depth] = pd.DataFrame(
            [per_class[test_depth, test_depth] - per_class[train_depth, test_depth]]
        ).T.describe(percentiles).T

table = pd.concat(rows)
table = table.droplevel(2)
table.index = table.index.set_names(['test', 'train'])
del table['count']
# table.format('{:+.1%}')

def highlight(s):
    if s.name == 'std':
        return None
    return ['color: green;' if x > 0 else 'color: black;' for x in s]

table.style.apply(highlight).format('{:+.1%}')

In [11]:
import infer

In [12]:
acc = {}

for train_depth in exp_dirs:
    print(train_depth)
    output_file = pathlib.Path(exp_dirs[train_depth]) / 'predictions/output-epoch-0020.pkl'
    outputs = np.load(output_file, allow_pickle=True)
    prob_file = pathlib.Path(exp_dirs[train_depth]) / 'predictions/prob-epoch-0020.npz'
    prob = np.load(prob_file, allow_pickle=False)

    train_subset = node_subsets[train_depth]
    train_projection = projections[train_depth]
    gt_train = train_subset[outputs['gt']]

    for test_depth in exp_dirs:
        if test_depth > train_depth:
            continue
        test_subset = node_subsets[test_depth]
        test_projection = projections[test_depth]
        
        is_leaf = subtrees[test_depth].leaf_mask()

        pred = []
        for i, prob_i in enumerate(prob.values()):
            pred_i = infer.argmax_where(prob_i[:, train_projection[test_subset]], is_leaf)
            pred.append(pred_i)
        pred = np.concatenate(pred, axis=0)

        gt = test_projection[gt_train]
        acc[train_depth, test_depth] = np.mean(gt == pred)

3
4
5
6
7


In [13]:
pd.Series(acc).unstack()

,3,4,5,6,7
3,0.82732,NaN,NaN,NaN,NaN
4,0.85242,0.60735,NaN,NaN,NaN
5,0.86614,0.64661,0.50681,NaN,NaN
6,0.86995,0.68298,0.56823,0.44626,NaN
7,0.87368,0.70484,0.59800,0.49295,0.40945


In [14]:
# Obtain *per-class* accuracy at each depth.

per_class = {}

for train_depth in exp_dirs:
    print(train_depth)
    output_file = pathlib.Path(exp_dirs[train_depth]) / 'predictions/output-epoch-0020.pkl'
    outputs = np.load(output_file, allow_pickle=True)
    prob_file = pathlib.Path(exp_dirs[train_depth]) / 'predictions/prob-epoch-0020.npz'
    prob = np.load(prob_file, allow_pickle=False)

    train_subset = node_subsets[train_depth]
    train_projection = projections[train_depth]
    gt_train = train_subset[outputs['gt']]

    for test_depth in exp_dirs:
        if test_depth > train_depth:
            continue
        test_subset = node_subsets[test_depth]
        test_projection = projections[test_depth]
        
        is_leaf = subtrees[test_depth].leaf_mask()

        pred = []
        for i, prob_i in enumerate(prob.values()):
            pred_i = infer.argmax_where(prob_i[:, train_projection[test_subset]], is_leaf)
            pred.append(pred_i)
        pred = np.concatenate(pred, axis=0)

        gt = test_projection[gt_train]

        is_correct = (gt == pred)
        per_class[train_depth, test_depth] = np.array([
            np.mean(is_correct[gt == y])
            for y in subtrees[test_depth].leaf_subset()
        ])

3
4
5
6
7


In [15]:
# Display per-class accuracy statistics.
#
# Note that there are 10 examples per leaf node.
# This makes the numbers round at deep levels.

rows = {}
for test_depth in exp_dirs:
    for train_depth in exp_dirs:
        if train_depth < test_depth:
            continue
        rows[test_depth, train_depth] = pd.DataFrame(
            [per_class[train_depth, test_depth]]
        ).T.describe().T

table = pd.concat(rows)
table = table.droplevel(2)
table.index = table.index.set_names(['test', 'train'])
del table['count']
# table.format('{:+.1%}')

table.style.format('{:.1%}')

In [16]:
# Look at difference in per-class accuracy between coarse and fine levels.

percentiles = [0.5, 0.85, 0.95]

rows = {}
for test_depth in exp_dirs:
    for train_depth in exp_dirs:
        if train_depth <= test_depth:
            continue
        rows[test_depth, train_depth] = pd.DataFrame(
            [per_class[test_depth, test_depth] - per_class[train_depth, test_depth]]
        ).T.describe(percentiles).T

table = pd.concat(rows)
table = table.droplevel(2)
table.index = table.index.set_names(['test', 'train'])
del table['count']
# table.format('{:+.1%}')

def highlight(s):
    if s.name == 'std':
        return None
    return ['color: green;' if x > 0 else 'color: black;' for x in s]

table.style.apply(highlight).format('{:+.1%}')